In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
from xgboost import XGBClassifier
# from sklearn import cross_validation, metrics
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

from xgboost import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 8

from numpy import sort
from sklearn.feature_selection import SelectFromModel

# # Ignore warnings
# import warnings
# warnings.simplefilter('ignore')

In [2]:
# Import the train set and test set
train_data2 = pd.read_csv("trainset.csv", delimiter=",")
test_data2 = pd.read_csv("testset.csv", delimiter=",")

train = pd.read_csv("train_rodolfo.csv", delimiter=",").fillna(0)
test = pd.read_csv("test_rodolfo.csv", delimiter=",").fillna(0)

#train = train.dropna()
# test = test.dropna()

FileNotFoundError: File b'train_rodolfo.csv' does not exist

In [ ]:
train.shape, test.shape

In [ ]:
target = 'Product'
IDcol = 'ID'

In [ ]:
#test_results = pd.read_csv('test_results.csv')
def modelfit2(alg, dtrain, dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    # fit model on training data
    model = alg
    
    X_train, X_test, y_train, y_test = train_test_split(dtrain[predictors], dtrain[target], test_size=0.33, random_state=7)
    eval_set = [(X_test, y_test)]
    model.fit(X_train,
              y_train,
              early_stopping_rounds=early_stopping_rounds,
              eval_metric="merror",
              eval_set=eval_set,
              verbose=False)
    
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    f1_scor = f1_score(y_test, predictions, average='micro')  
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("f1_micro: %.2f%%" % (f1_scor * 100.0))
    
#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
#         xgtest = xgb.DMatrix(dtest[predictors].values)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
#             metrics='merror',
#                           early_stopping_rounds=early_stopping_rounds)
#         alg.set_params(n_estimators=cvresult.shape[0])
    
#     #Fit the algorithm on the data
#     alg.fit(dtrain[predictors], dtrain['Product'],eval_metric='auc')
        
#     #Predict training set:
#     dtrain_predictions = alg.predict(dtrain[predictors])
#     dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
#     #Print model report:
#     print("\nModel Report")
#     print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Product'].values, dtrain_predictions))
#     #print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Product'], dtrain_predprob))
    
# #     Predict on testing data:
    dtest['predprob'] = alg.predict_proba(dtest[predictors])[:,1]
    results = test_results.merge(dtest[['ID','predprob']], on='ID')
    #print(dtest[['ID','predprob']])
#     #print('AUC Score (Test): %f' % metrics.roc_auc_score(results['Product'], results['predprob']))
        
    #feat_imp = pd.Series(model.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

    plot_importance(model)
    plt.show()
    print("Model: " + str(model))
    thresholds = alg.feature_importances_
    print("Feature_Importances: ")
    print(dtrain.columns[thresholds.argsort()[-10:][::-1]])

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='merror', early_stopping_rounds=early_stopping_rounds, 
                          verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Product'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    #dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Product'].values, dtrain_predictions))
    print("F1_score : %f" % metrics.f1_score(dtrain['Product'], dtrain_predictions, average='micro'))
                    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')
    plot_importance(alg)
    plt.show()
    thresholds = alg.feature_importances_
    print("Feature_Importances: ")
    print(dtrain.columns[thresholds.argsort()[-10:][::-1]])
    print("Model: " + str(alg.get_xgb_params()))
    print("n_estimator: " + str(cvresult.shape[0]))
    return cvresult.shape[0]

In [ ]:
def important_features(model, nfeatures = 10):
    model.fit()
    thresholds = model.feature_importances_
    
    return train.column[thresholds.argsort()[-nfeatures:][::-1]]

# Step 1- Find the number of estimators for a high learning rate

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb0 = XGBClassifier(
        objective='multi:softprob',
        num_class=4)
# print(train[predictors].values)
modelfit(xgb0, train, predictors)

In [ ]:
learning_rate = 0.1
n_estimators = 1000
max_depth = 5
min_child_weight = 1
gamma = 0
subsample = 0.8
colsample_bytree = 0.8
reg_alpha = 0.005
cv = 5

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        objective='multi:softprob',
        num_class=4,
        nthread=4,
        scale_pos_weight=1,
        seed=27)
n_estimators = modelfit(xgb1, train, predictors)

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample,
                                        colsample_bytree=colsample_bytree, objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='f1_micro',n_jobs=4,iid=False, cv=cv,verbose=1)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
max_depth = gsearch1.best_params_['max_depth']
min_child_weight = gsearch1.best_params_['min_child_weight']
max_depth,min_child_weight

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[max_depth-1,max_depth,max_depth+1],
    'min_child_weight':[min_child_weight-1,min_child_weight,min_child_weight+1]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='f1_micro',n_jobs=4,iid=False, cv=cv, verbose=1)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
max_depth = gsearch2.best_params_['max_depth']
min_child_weight = gsearch2.best_params_['min_child_weight']
max_depth,min_child_weight

In [ ]:
#Grid seach on subsample and max_features#Grid se 
#Choose all predictors except target & IDcols
param_test2b = {
    'min_child_weight':[min_child_weight,min_child_weight+2,min_child_weight+4,min_child_weight+6]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='f1_micro',n_jobs=4,iid=False, cv=cv, verbose=1)
gsearch2b.fit(train[predictors],train[target])
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

In [ ]:
min_child_weight = gsearch2b.best_params_['min_child_weight']
min_child_weight

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='f1_micro',n_jobs=4,iid=False, cv=cv, verbose=1)
gsearch3.fit(train[predictors],train[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
gamma = gsearch3.best_params_['gamma']
gamma

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb2 = XGBClassifier(
        learning_rate =learning_rate,
        n_estimators=1000,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        objective='multi:softprob', 
        num_class=4,
        nthread=4,
        scale_pos_weight=1,
        seed=27)
n_estimators = modelfit(xgb2, train, predictors)

# Tune subsample and colsample_bytree

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='f1_micro',n_jobs=4,iid=False, cv=cv, verbose=1)
gsearch4.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
subsample = gsearch4.best_params_['subsample']
colsample_bytree = gsearch4.best_params_['colsample_bytree']
subsample,colsample_bytree

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test5 = {
    'subsample':[i/100.0 for i in range(int(subsample*100-5),int(subsample*100+5),5)],
    'colsample_bytree':[i/100.0 for i in range(int(colsample_bytree*100-5),int(colsample_bytree*100+5),5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test5, scoring='f1_micro',n_jobs=4,iid=False, cv=cv, verbose=1)
gsearch5.fit(train[predictors],train[target])
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

In [ ]:
subsample = gsearch5.best_params_['subsample']
colsample_bytree = gsearch5.best_params_['colsample_bytree']
subsample,colsample_bytree

# regularization:

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='f1_micro',n_jobs=4,iid=False, cv=cv, verbose=1)
gsearch6.fit(train[predictors],train[target])
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [ ]:
reg_alpha = gsearch6.best_params_['reg_alpha']
reg_alpha

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test7 = {
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test7, scoring='f1_micro',n_jobs=4,iid=False, cv=cv,verbose=1)
gsearch7.fit(train[predictors],train[target])
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

In [ ]:
reg_alpha = gsearch7.best_params_['reg_alpha']
reg_alpha

In [ ]:
xgb3 = XGBClassifier(
        learning_rate =learning_rate,
        n_estimators=1000,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        objective='multi:softprob', 
        num_class=4,
        nthread=4,
        scale_pos_weight=1,
        seed=27)
n_estimators = modelfit(xgb3, train, predictors)

In [ ]:
xgb4 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=209,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        objective='multi:softprob', 
        num_class=4,
        nthread=4,
        scale_pos_weight=1,
        seed=27)
n_estimators = modelfit(xgb4, train, predictors)

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test8 = {
    #learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
    'n_estimators' : [50, 100, 150, 200, 300]
}
gsearch8 = GridSearchCV(estimator = XGBClassifier( learning_rate =learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                                        min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree,
                                        objective='multi:softprob', num_class=4, nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test8, scoring='f1_micro',n_jobs=4,iid=False, cv=cv,verbose=1)
gsearch8.fit(train[predictors],train[target])
gsearch8.grid_scores_, gsearch8.best_params_, gsearch8.best_score_

In [ ]:
train.columns

In [ ]:
train[predictors].shape,train[target].shape,test[predictors].shape

In [ ]:
pred = ['ZipCode', 'CustomerExpatriate', 'MonthlySMSTrafficCount',
       'DeviceFlagSmartphone', 'DeviceFlag4G', 'DataArpu', 'Region',
       'MonthlyVoiceTrafficCount', 'Instant-Messaging-Applications',
       'Unclassified']
train[pred].head()

In [ ]:
xgb4 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        objective='multi:softprob', 
        num_class=4,
        nthread=4,
        scale_pos_weight=1,
        seed=27)
n_estimators = modelfit(xgb4, train, predictors)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(train[predictors], train[target], test_size=0.33, random_state=7)
# eval_set = [(X_test, y_test)]
xgb4.fit(train[pred],
      train[target],
      eval_metric="merror",
      verbose=True)

y_pred_new_data = xgb4.predict(test[pred])
predictions = [round(value) for value in y_pred_new_data]

print("F1 score on unseen data: %.3f%%" % (f1_score(train[target][0:3190], y_pred_new_data, average='micro')*100))

In [ ]:
pd.DataFrame({'Product': y_pred_new_data}).describe(),pd.DataFrame({'Product': predictions}).describe()

In [ ]:
y = pd.DataFrame({'Product': y_pred_new_data})
submit = test['ID']
submit = pd.DataFrame({'ID': submit})

pred = y
pred = pd.DataFrame({'Product': y_pred_new_data})

product = []
product = pred.Product
product.replace({ 
    int(0) : 'Non-Customer', 
    int(1) : 'V-Bag', 
    int(2) : 'V-Pet', 
    int(3) : 'V-Auto'},
    inplace=True)

submit = submit.join(product)
submit

In [ ]:
submit.Product.describe()

In [ ]:
submit.to_csv('resultxgb.csv', index = False, encoding='utf-8')

In [ ]:
# predictors = [x for x in train.columns if x not in [target]]
# # split data into X and y
# X = train[predictors]
# Y = train[target]
# # split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
# # fit model on all training data
# model = XGBClassifier(
#         objective='multi:softprob', 
#         num_class=4)
# model.fit(X_train, y_train)
# # make predictions for test data and evaluate
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# # Fit model using each importance as a threshold
# thresholds = sort(model.feature_importances_)
# for thresh in thresholds:
#     # select features using threshold
#     selection = SelectFromModel(model, threshold=thresh, prefit=True)
#     select_X_train = selection.transform(X_train)
#     # train model
#     selection_model = XGBClassifier(objective='multi:softprob', 
#         num_class=4)
#     selection_model.fit(select_X_train, y_train)
#     # eval model
#     select_X_test = selection.transform(X_test)
#     y_pred = selection_model.predict(select_X_test)
#     predictions = [round(value) for value in y_pred]
#     accuracy = accuracy_score(y_test, predictions)
#     print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [ ]:
# thresholds = model.feature_importances_
# thresholds

# thresholds.argsort()[-10:][::-1]
# train.columns[thresholds.argsort()[-10:][::-1]]